In [ ]:
import sys
sys.path.append("..")

from models import Yolov4
from tensorflow import keras
from config import yolo_config    # 要記得改!!!!!!
from utils import DataGenerator, read_annotation_lines

import pickle
from glob import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import os
# 使用第一張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# import random
# seed = 500
# tf.random.set_seed(500)
# np.random.seed(seed)
# random.seed(seed)

In [ ]:
#22 1102是沒有10%背
#22 1104是有10%背景

#186 0330是沒有10%背
#186 0331是有10%背景
train_lines = read_annotation_lines('../dataset/txt/20230331_anno_train_lines_0.txt')   
val_lines   = read_annotation_lines('../dataset/txt/20230331_anno_val_lines_0.txt')

In [ ]:
FOLDER_PATH     = '../dataset/img/20230331_train_vali_img_sopbox_0'         # image位置
class_name_path = '../class_names/classes.txt'                              # class位置
data_gen_train  = DataGenerator(train_lines, class_name_path, FOLDER_PATH)  
data_gen_vali   = DataGenerator(val_lines, class_name_path, FOLDER_PATH)

model = Yolov4(weight_path=None, class_name_path=class_name_path)

In [ ]:
print('訓練集數量= ', len(train_lines),'驗證集數量= ',len(val_lines))

# 訓練自己的模型

In [ ]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []
        
#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

# history = LossHistory()
''''''''''''''''''''''''''''''''''''''
day = '20230404'
checkpoint_filepath = '../model/202303/{}_sopbox_0.weights'.format(day)
val_loss_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

## TODO:
val_mAP_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='auc',
    mode='max',
    save_best_only=True
    )


callbacks = [val_loss_checkpoint_callback, 
            #  val_mAP_checkpoint_callback
            ]
            
epochs    = 5
his = model.fit(train_data_gen=data_gen_train, 
                initial_epoch = 0,
                epochs       = epochs, 
                val_data_gen = data_gen_vali,
                callbacks    = callbacks,
          )

# 畫圖

In [ ]:
loss     = his.history['loss']
val_loss = his.history['val_loss']

In [ ]:
x = [i for i in range(1, epochs+1)]

min_valloss_x = val_loss.index(min(val_loss)) + 1
min_valloss_y = min(val_loss)

plt.figure(figsize=(24,4))
plt.plot(x, loss, 'r')     
plt.plot(x, val_loss, 'b')     # red dotted line (no marker)

plt.plot(min_valloss_x, min_valloss_y, 'd', color='g')
plt.text(min_valloss_x, min_valloss_y, "({},{})".format(min_valloss_x,round(min_valloss_y,1)), ha='left',va='top',fontsize=20)

plt.legend(['loss','val_loss'])
plt.ylim((0, 1000))
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('loss')
plt.savefig('../model/202302/{}_loss.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
with open('../model/202303/{}_val_loss.list'.format(day),'wb') as f:
    pickle.dump(val_loss,f) 

In [ ]:
with open('../model/202303/{}_loss.list'.format(day),'wb') as f:
    pickle.dump(loss,f) 

In [ ]:
with open('../model/202303/{}_loss.his'.format(day),'wb') as f:
    pickle.dump(his,f) 